In [1]:
import os
import re

files = os.listdir("../tmp/")

In [2]:
def get_pibt_params(file):
    r = file.split(".")
    props = {"scenario": r[0],  "num_agents": int(r[-2])}
    with open(os.path.join("../tmp", file)) as f:
        for line in f:
            if "solved:" in line:
                if "True" in line:
                    props["Solved"] = True
                else:
                    props["Solved"] = False
            if "after smoothing" in line:
                length = line.strip().split(":")[1]
                props["Length"]= float(length)
            if "PiBT" in line:
                props["Pibt solve time"] = float(re.findall("\d+\.\d+", line)[0])
            if "Smoothing" in line:
                props["Solve time"] = float(re.findall("\d+\.\d+", line)[0])
            if "%" in line:
                score = line.strip().split(":")[1]
                props["Reduction"] = float(score.replace("%", ""))
    return props

<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
<>:15: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_2234537/2070378861.py:15: SyntaxWarning: invalid escape sequence '\d'
  props["Pibt solve time"] = float(re.findall("\d+\.\d+", line)[0])
/tmp/ipykernel_2234537/2070378861.py:17: SyntaxWarning: invalid escape sequence '\d'
  props["Solve time"] = float(re.findall("\d+\.\d+", line)[0])


In [3]:
import math

def count_collisions_at_end_implies_final(trajectory_list):
    """
    Counts collisions between agents, ignoring collisions where one agent
    is at its final location (the last point in its trajectory).

    Args:
        trajectory_list (dict): A dictionary where keys are agent IDs and
                                 values are lists of (x, y) coordinate tuples
                                 representing their trajectories. The last point
                                 in each trajectory is assumed to be the final location.

    Returns:
        int: The total number of collisions detected (excluding those with
             agents at their final locations).
    """
    agents = list(trajectory_list.keys())
    num_agents = len(agents)
    collision_count = 0

    def distance(p1, p2):
        """Calculates the Euclidean distance between two points."""
        return math.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

    # Iterate through all unique pairs of agents
    for i in range(num_agents):
        for j in range(i + 1, num_agents):
            agent1_id = agents[i]
            agent2_id = agents[j]
            trajectory1 = trajectory_list[agent1_id]
            trajectory2 = trajectory_list[agent2_id]
            end_loc1 = trajectory1[-1] if trajectory1 else None
            end_loc2 = trajectory2[-1] if trajectory2 else None

            # Check for collision at each corresponding time step
            min_len = min(len(trajectory1), len(trajectory2))
            for k in range(min_len):
                pos1 = trajectory1[k]
                pos2 = trajectory2[k]

                # Check if either agent is at their final location
                agent1_at_end = (end_loc1 is not None and pos1[0] == end_loc1[0] and pos1[1] == end_loc1[1])
                agent2_at_end = (end_loc2 is not None and pos2[0] == end_loc2[0] and pos2[1] == end_loc2[1])

                if not (agent1_at_end or agent2_at_end) and distance(pos1, pos2) <= 1:
                    print()
                    collision_count += 1
                    break  # Count each pair's collision only once

    return collision_count

# Example usage:
trajectories = {
    "agent1": [(0, 0), (1, 1), (2, 2)],
    "agent2": [(0.5, 0.5), (1.2, 1.2), (2.1, 2.1)],
    "agent3": [(5, 5), (6, 6), (7, 7)],
    "agent4": [(1, 0), (1, 1), (1, 2)],
    "agent5": [(0, 1), (1, 1)],  # Different length trajectory
}

collisions = count_collisions_at_end_implies_final(trajectories)
print(f"Number of collisions (ignoring agents at final locations): {collisions}")






Number of collisions (ignoring agents at final locations): 5


In [4]:
pibt_results = [file for file in files if "pibt" in file]
pibt_results

['lak303d-random-4.scen.pibt.4.yaml',
 'random-32-32-10-random-20.scen.pibt.20.yaml',
 'maze-32-32-2-random-20.scen.pibt.2.yaml',
 'empty-32-32-random-2.scen.pibt.20.yaml',
 'maze-32-32-4-random-21.scen.pibt.15.yaml',
 'den520d-random-21.scen.pibt.20.yaml',
 'empty-16-16-random-18.scen.pibt.4.yaml',
 'ost003d-random-3.scen.pibt.15.yaml',
 'den312d-random-21.scen.pibt.2.yaml',
 'random-32-32-10-random-2.scen.pibt.100.yaml',
 'empty-8-8-random-10.scen.pibt.10.yaml',
 'ht_mansion_n-random-1.scen.pibt.4.yaml',
 'room-64-64-8-random-2.scen.pibt.100.yaml',
 'brc202d-random-7.scen.pibt.100.yaml',
 'warehouse-20-40-10-2-2-random-12.scen.pibt.2.yaml',
 'random-32-32-20-random-6.scen.pibt.10.yaml',
 'empty-8-8-random-14.scen.pibt.10.yaml',
 'lak303d-random-24.scen.pibt.10.yaml',
 'random-32-32-20-random-11.scen.pibt.100.yaml',
 'warehouse-20-40-10-2-2-random-5.scen.pibt.4.yaml',
 'empty-16-16-random-16.scen.pibt.15.yaml',
 'random-64-64-20-random-23.scen.pibt.100.yaml',
 'random-32-32-20-random-

In [5]:
p = [get_pibt_params(pibt) for pibt in pibt_results]

In [6]:
import pandas as pd

p = pd.DataFrame(p)
p.iloc[[p["Reduction"].idxmax()]]

,scenario,num_agents,Solved,Length,Reduction,Pibt solve time,Solve time
1081,Berlin_1_256-random-18,2,True,173.305494,45.359431,0.4116,0.4297


In [7]:
p[p["scenario"] == "Berlin_1_256-random-18"]

,scenario,num_agents,Solved,Length,Reduction,Pibt solve time,Solve time
303,Berlin_1_256-random-18,15,True,2343.433731,14.287962,4.2062,4.3652
538,Berlin_1_256-random-18,4,True,442.319926,27.803484,0.8186,0.8561
850,Berlin_1_256-random-18,20,True,3014.223781,12.943854,5.2770,5.4972
1081,Berlin_1_256-random-18,2,True,173.305494,45.359431,0.4116,0.4297
2778,Berlin_1_256-random-18,10,True,1481.081885,15.057697,2.6971,2.7939
2891,Berlin_1_256-random-18,100,True,17085.063996,10.090044,27.2913,29.6621


In [8]:
p.iloc[[p["Reduction"].idxmin()]]

,scenario,num_agents,Solved,Length,Reduction,Pibt solve time,Solve time
3890,warehouse-20-40-10-2-1-random-6,2,True,290.0,-1.960118e-14,0.2426,0.26


In [9]:
p[p["scenario"] == "warehouse-20-40-10-2-1-random-6"]

,scenario,num_agents,Solved,Length,Reduction,Pibt solve time,Solve time
1404,warehouse-20-40-10-2-1-random-6,4,True,658.014835,2.410350e+00,0.4905,0.5296
2016,warehouse-20-40-10-2-1-random-6,20,True,3706.929812,1.922358e+00,2.4731,2.7914
2600,warehouse-20-40-10-2-1-random-6,10,True,1970.208868,1.968869e+00,1.4309,1.5766
3559,warehouse-20-40-10-2-1-random-6,100,False,18239.661352,1.408864e+00,13.5710,17.5644
3653,warehouse-20-40-10-2-1-random-6,15,True,3043.343144,2.141427e+00,2.0483,2.2809
3890,warehouse-20-40-10-2-1-random-6,2,True,290.000000,-1.960118e-14,0.2426,0.2600


In [10]:
def get_ccbs_params(file):
    r = file.split(".")
    props = {"scenario": r[0],  "num_agents": int(r[-3])}
    with open(os.path.join("../tmp", file)) as f:
        for line in f:
            if "Total length:" in line:
                d = re.findall("\d+\.\d+", line)
                if len(d) > 0:
                    props["Length"] = float(d[0])
                else:
                    props["Length"] = float("NaN") 
                props["Success"] = True
            else:
                props["Length"] = float("NaN")
                props["Success"] = False

            if "seconds" in line:
                props["Time"] = float(re.findall("\d+\.\d+", line)[0])
    return props
        

<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:18: SyntaxWarning: invalid escape sequence '\d'
/tmp/ipykernel_2234537/1227642783.py:7: SyntaxWarning: invalid escape sequence '\d'
  d = re.findall("\d+\.\d+", line)
/tmp/ipykernel_2234537/1227642783.py:18: SyntaxWarning: invalid escape sequence '\d'
  props["Time"] = float(re.findall("\d+\.\d+", line)[0])


In [11]:
ccbs_results = [file for file in files if "res" in file]
ccbs_results

['den312d-random-25.scen.2.yaml.res',
 'room-32-32-4-random-5.scen.15.yaml.res',
 'ost003d-random-20.scen.2.yaml.res',
 'maze-128-128-10-random-3.scen.10.yaml.res',
 'Berlin_1_256-random-9.scen.10.yaml.res',
 'den520d-random-25.scen.10.yaml.res',
 'maze-128-128-2-random-12.scen.20.yaml.res',
 'Paris_1_256-random-16.scen.20.yaml.res',
 'random-64-64-20-random-16.scen.15.yaml.res',
 'warehouse-20-40-10-2-2-random-23.scen.4.yaml.res',
 'w_woundedcoast-random-3.scen.20.yaml.res',
 'maze-128-128-1-random-21.scen.10.yaml.res',
 'den520d-random-5.scen.15.yaml.res',
 'maze-128-128-10-random-5.scen.15.yaml.res',
 'empty-16-16-random-3.scen.4.yaml.res',
 'random-32-32-20-random-2.scen.4.yaml.res',
 'Boston_0_256-random-15.scen.15.yaml.res',
 'warehouse-10-20-10-2-1-random-22.scen.4.yaml.res',
 'warehouse-20-40-10-2-2-random-11.scen.20.yaml.res',
 'Boston_0_256-random-23.scen.10.yaml.res',
 'lak303d-random-14.scen.20.yaml.res',
 'random-64-64-10-random-6.scen.20.yaml.res',
 'Berlin_1_256-random-8

In [12]:
c = [get_ccbs_params(ccbs) for ccbs in ccbs_results]
c = pd.DataFrame(c)
successful_ccbs = c[c["Success"]]
merged = pd.merge(successful_ccbs, p, on=["scenario", "num_agents"], how='inner').sort_values(["scenario", "num_agents"])
merged

,scenario,num_agents,Length_x,Success,Time,Solved,Length_y,Reduction,Pibt solve time,Solve time
1541,Berlin_1_256-random-1,2,205.083261,True,0.277712,True,192.100919,21.432601,0.2792,0.2945
526,Berlin_1_256-random-1,4,361.994949,True,0.508663,True,347.011751,19.046169,0.4081,0.4336
1249,Berlin_1_256-random-16,2,193.911688,True,0.330278,True,208.152968,9.641116,0.3218,0.3367
505,Berlin_1_256-random-18,2,273.124892,True,0.656679,True,173.305494,45.359431,0.4116,0.4297
207,Berlin_1_256-random-20,2,174.911688,True,0.280080,True,178.363381,12.984009,0.2892,0.3072
...,...,...,...,...,...,...,...,...,...,...
1414,warehouse-20-40-10-2-2-random-6,4,593.965512,True,1.161931,True,650.299366,4.191530,0.8176,0.8659
1593,warehouse-20-40-10-2-2-random-7,2,291.769553,True,0.523454,True,290.996136,13.523880,0.3410,0.3651
691,warehouse-20-40-10-2-2-random-7,4,594.651804,True,1.001817,True,614.069174,10.475223,0.6966,0.7426
1451,warehouse-20-40-10-2-2-random-8,2,231.012193,True,0.454738,True,271.605122,0.000000,0.3243,0.3498


In [13]:
merged[merged["scenario"] == "Berlin_1_256-random-20"]

,scenario,num_agents,Length_x,Success,Time,Solved,Length_y,Reduction,Pibt solve time,Solve time
207,Berlin_1_256-random-20,2,174.911688,True,0.28008,True,178.363381,12.984009,0.2892,0.3072


In [14]:
p[p["scenario"] == "warehouse-20-40-10-2-1-random-3"]

,scenario,num_agents,Solved,Length,Reduction,Pibt solve time,Solve time
126,warehouse-20-40-10-2-1-random-3,10,True,1960.635310,0.617904,1.4300,1.5570
2646,warehouse-20-40-10-2-1-random-3,4,True,735.107889,0.257698,0.4934,0.5477
2770,warehouse-20-40-10-2-1-random-3,2,True,256.410539,0.620767,0.1474,0.1655
2947,warehouse-20-40-10-2-1-random-3,20,True,3034.188470,0.889917,2.2293,2.4890
3595,warehouse-20-40-10-2-1-random-3,15,True,2625.399570,0.754985,1.9274,2.1166
4288,warehouse-20-40-10-2-1-random-3,100,True,16336.808379,1.315663,11.2189,13.4938


In [15]:
((merged["Length_y"] - merged["Length_x"]) / merged["Length_x"]).mean()

0.0083227139262335

In [16]:
merged[merged["num_agents"] == 15]

,scenario,num_agents,Length_x,Success,Time,Solved,Length_y,Reduction,Pibt solve time,Solve time
1409,den312d-random-11,15,719.404112,True,0.808179,True,799.419785,4.455353,0.2003,0.2435
602,den312d-random-2,15,573.119841,True,0.573352,True,616.285390,6.726889,0.1748,0.2201
1678,den312d-random-22,15,746.404112,True,0.836808,True,828.268152,4.586459,0.2120,0.2734
608,den312d-random-4,15,573.007143,True,0.363055,True,634.205601,6.570030,0.1762,0.2326
654,den312d-random-6,15,668.475180,True,2.828310,True,733.149283,5.471437,0.1928,0.2406
...,...,...,...,...,...,...,...,...,...,...
1035,warehouse-20-40-10-2-1-random-3,15,2591.867099,True,2.908801,True,2625.399570,0.754985,1.9274,2.1166
1004,warehouse-20-40-10-2-1-random-5,15,2632.374675,True,2.219431,True,2719.209332,1.435001,2.1372,2.3435
1012,warehouse-20-40-10-2-1-random-7,15,2733.276262,True,2.140227,True,2729.215787,1.744355,1.9989,2.2048
1873,warehouse-20-40-10-2-1-random-8,15,2917.872150,True,2.755853,True,2931.358132,2.171666,2.1070,2.3132


In [ ]:
c.to_csv("results.aaccbs.csv")

In [18]:
p.to_csv("results.pibt.csv")